# Word Vectorization - Lab

## Introduction

In this lab, you'll tokenize and vectorize text documents, create and use a bag of words, and identify words unique to individual documents using TF-IDF vectorization. 


## Objectives

In this lab you will:  

- Implement tokenization and count vectorization from scratch 
- Implement TF-IDF from scratch 
- Use dimensionality reduction on vectorized text data to create and interpret visualizations 

## Let's get started!

Run the cell below to import everything necessary for this lab.  

In [3]:
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt', quiet=True)
np.random.seed(0)
import os

### Our Corpus

In this lab, we'll be working with 20 different documents, each containing song lyrics from either Garth Brooks or Kendrick Lamar albums.  

The songs are contained within the `data` subdirectory, contained within the same folder as this lab.  Each song is stored in a single file, with files ranging from `song1.txt` to `song20.txt`.  

To make it easy to read in all of the documents, use a list comprehension to create a list containing the name of every single song file in the cell below. 

In [4]:
current_dir = os.getcwd()

# Construct the path to the 'data' subdirectory
data_dir = os.path.join(current_dir, 'data')

# Create a list comprehension to get all song filenames
filenames = [f'song{i}.txt' for i in range(1, 21)]
filenames

['song1.txt',
 'song2.txt',
 'song3.txt',
 'song4.txt',
 'song5.txt',
 'song6.txt',
 'song7.txt',
 'song8.txt',
 'song9.txt',
 'song10.txt',
 'song11.txt',
 'song12.txt',
 'song13.txt',
 'song14.txt',
 'song15.txt',
 'song16.txt',
 'song17.txt',
 'song18.txt',
 'song19.txt',
 'song20.txt']

Next, let's import a single song to see what our text looks like so that we can make sure we clean and tokenize it correctly. 

Use the code in the cell below to read in the lyrics from `song18.txt` as a list of lines, just using vanilla Python:

In [34]:
# Import a
with open('data/song18.txt') as f:
    test_song = f.readlines()
    
test_song


['[Kendrick Lamar:]\n',
 "Two wrongs don't make us right away\n",
 "Tell me something's wrong\n",
 'Party all of our lives away\n',
 'To take you on\n',
 '[Zacari:]\n',
 'Oh, baby I want you\n',
 'Baby I need you\n',
 'I wanna see you\n',
 'Baby I wanna go out yeah\n',
 'Baby I wanna go out yeah\n',
 'Baby I want you\n',
 'Baby I need you\n',
 'I wanna see you\n',
 'Baby I wanna go out yeah\n',
 'Baby I wanna go out yeah\n',
 'All night (all night, all night)\n',
 'All night\n',
 "Your body's on fire\n",
 'And your drinks on ice\n',
 'All night (all night, all night)\n',
 'All night\n',
 "Your body's on fire\n",
 'And your drinks on ice\n',
 '[Babes Wodumo:]\n',
 'Oh my word oh my gosh oh my word (Oh my gosh)\n',
 'Oh my word oh my gosh oh my word (Oh my gosh)\n',
 'Oh my word oh my gosh oh my word (Oh my gosh)\n',
 'Oh my word oh my gosh oh my word (Oh my gosh)\n',
 'Everybody say kikiritikiki (kikiritikiki)\n',
 'Everybody say kikiritikiki (kikiritikiki)\n',
 'Everybody say kikiritik

### Tokenizing our Data

Before we can create a bag of words or vectorize each document, we need to clean it up and split each song into an array of individual words.

Consider the following sentences from the example above:

`"Two wrongs don't make us right away\n", "Tell me something's wrong\n"`

After tokenization, this should look like:

`['two', 'wrongs', 'dont', 'make', 'us', 'right', 'away', 'tell', 'me', 'somethings', 'wrong']`

Tokenization is pretty tedious if we handle it manually, and would probably make use of regular expressions, which is outside the scope of this lab. In order to keep this lab moving, we'll use a library function to clean and tokenize our data so that we can move onto vectorization.  

Tokenization is a required task for just about any Natural Language Processing (NLP) task, so great industry-standard tools exist to tokenize things for us, so that we can spend our time on more important tasks without getting bogged down hunting every special symbol or punctuation in a massive dataset. For this lab, we'll make use of the tokenizer in the amazing `nltk` library, which is short for _Natural Language Tool Kit_.

**_NOTE:_** NLTK requires extra installation methods to be run the first time certain methods are used.  If `nltk` throws you an error about needing to install additional packages, follow the instructions in the error message to install the dependencies, and then rerun the cell.  

Before we tokenize, however, we need to do one more step! Computers are very particular about strings. If we tokenized our data in its current state, we would run into the following problems:

- Counting things that aren't actually words.  In the example above, `"[Kendrick Lamar:]"` is a note specifying who is speaking, not a lyric contained in the actual song, so it should be removed. 
- Punctuation and capitalization would mess up our word counts. To the Python interpreter, `all`, `All`, and `(all` are unique words, and would all be counted separately.  We need to remove punctuation and capitalization, so that all words will be counted correctly. 

Before we tokenize our songs, we'll do only a small manual bit of cleaning. 

In the cell below, write a function to:
- remove lines that just contain `['artist names']`
- join the list of strings into one big string for the entire song
- remove newline characters `\n`
- remove the following punctuation marks: `",.'?!()"`
- make every word lowercase

Test the function on `test_song` to show that it has successfully removed `'[Kendrick Lamar:]'` and other instances of artist names from the song, and is returning the song as one string (NOT a list of strings) with newlines (`\n`) and punctuation removed and every word in lowercase.

In [22]:
import re

In [23]:
def clean_song(song):
    artist_tag = re.compile(r"^\[.*?\]:?\s*$", re.IGNORECASE)

    # Step 1: Filter out lines that are *only* artist tags.
    remove_lines = []
    for line in song:
        # If the line (stripped of leading/trailing whitespace) does NOT match the artist tag pattern
        if not artist_tag.match(line.strip()):
            remove_lines.append(line)

    joined_lines = ' '.join(remove_lines)
    joined_lines = joined_lines.replace('\n', ' ')
    punctuation_to_remove = ",.'?!()"
    for char in punctuation_to_remove:
        joined_lines = joined_lines.replace(char, '')
    joined_lines = joined_lines.lower()
    joined_lines = ' '.join(joined_lines.split())

    return joined_lines


clean_test_song = clean_song(test_song)
print(clean_test_song)

two wrongs dont make us right away tell me somethings wrong party all of our lives away to take you on oh baby i want you baby i need you i wanna see you baby i wanna go out yeah baby i wanna go out yeah baby i want you baby i need you i wanna see you baby i wanna go out yeah baby i wanna go out yeah all night all night all night all night your bodys on fire and your drinks on ice all night all night all night all night your bodys on fire and your drinks on ice oh my word oh my gosh oh my word oh my gosh oh my word oh my gosh oh my word oh my gosh oh my word oh my gosh oh my word oh my gosh oh my word oh my gosh oh my word oh my gosh everybody say kikiritikiki kikiritikiki everybody say kikiritikiki kikiritikiki everybody say kikiritikiki kikiritikiki everybody say kikiritikiki kikiritikiki ungbambe ungdedele ungbhasobhe unggudluke ungbambe ungdedele ungbhasobhe unggudluke ungbambe ungdedele ungbhasobhe unggudluke ungbambe ungdedele ungbhasobhe unggudluke baby i want you baby i need yo

Great! Now, we can use `nltk`'s `word_tokenize()` function on the song string to get a fully tokenized version of the song. Test this function on `clean_test_song` to ensure that the function works. 

In [24]:
tokenized_test_song = word_tokenize(clean_test_song)

tokenized_test_song

['two',
 'wrongs',
 'dont',
 'make',
 'us',
 'right',
 'away',
 'tell',
 'me',
 'somethings',
 'wrong',
 'party',
 'all',
 'of',
 'our',
 'lives',
 'away',
 'to',
 'take',
 'you',
 'on',
 'oh',
 'baby',
 'i',
 'want',
 'you',
 'baby',
 'i',
 'need',
 'you',
 'i',
 'wan',
 'na',
 'see',
 'you',
 'baby',
 'i',
 'wan',
 'na',
 'go',
 'out',
 'yeah',
 'baby',
 'i',
 'wan',
 'na',
 'go',
 'out',
 'yeah',
 'baby',
 'i',
 'want',
 'you',
 'baby',
 'i',
 'need',
 'you',
 'i',
 'wan',
 'na',
 'see',
 'you',
 'baby',
 'i',
 'wan',
 'na',
 'go',
 'out',
 'yeah',
 'baby',
 'i',
 'wan',
 'na',
 'go',
 'out',
 'yeah',
 'all',
 'night',
 'all',
 'night',
 'all',
 'night',
 'all',
 'night',
 'your',
 'bodys',
 'on',
 'fire',
 'and',
 'your',
 'drinks',
 'on',
 'ice',
 'all',
 'night',
 'all',
 'night',
 'all',
 'night',
 'all',
 'night',
 'your',
 'bodys',
 'on',
 'fire',
 'and',
 'your',
 'drinks',
 'on',
 'ice',
 'oh',
 'my',
 'word',
 'oh',
 'my',
 'gosh',
 'oh',
 'my',
 'word',
 'oh',
 'my',
 'gos

Great! Now that we can tokenize our songs, we can move onto vectorization. 


### Count Vectorization

Machine Learning algorithms don't understand strings. However, they do understand math, which means they understand vectors and matrices.  By **_Vectorizing_** the text, we just convert the entire text into a vector, where each element in the vector represents a different word. The vector is the length of the entire vocabulary -- usually, every word that occurs in the English language, or at least every word that appears in our corpus.  Any given sentence can then be represented as a vector where all the vector is 1 (or some other value) for each time that word appears in the sentence. 

Consider the following example: 

<center>"I scream, you scream, we all scream for ice cream."</center>

| 'aardvark' | 'apple' | [...] | 'I' | 'you' | 'scream' | 'we' | 'all' | 'for' | 'ice' | 'cream' | [...] | 'xylophone' | 'zebra' |
|:----------:|:-------:|:-----:|:---:|:-----:|:--------:|:----:|:-----:|:-----:|:-----:|:-------:|:-----:|:-----------:|:-------:|
|      0     |    0    |   0   |  1  |   1   |     3    |   1  |   1   |   1   |   1   |    1    |   0   |      0      |    0    |

This is called a **_Sparse Representation_**, since the strong majority of the columns will have a value of 0.  Note that elements corresponding to words that do not occur in the sentence have a value of 0, while words that do appear in the sentence have a value of 1 (or 1 for each time it appears in the sentence).

Alternatively, we can represent this sentence as a plain old Python dictionary of word frequency counts:

```python
BoW = {
    'I':1,
    'you':1,
    'scream':3,
    'we':1,
    'all':1,
    'for':1,
    'ice':1,
    'cream':1
}
```

Both of these are examples of **_Count Vectorization_**. They allow us to represent a sentence as a vector, with each element in the vector corresponding to how many times that word is used.

#### Positional Information and Bag of Words

Notice that when we vectorize a sentence this way, we lose the order that the words were in.  This is the **_Bag of Words_** approach mentioned earlier.  Note that sentences that contain the same words will create the same vectors, even if they mean different things -- e.g. `'cats are scared of dogs'` and `'dogs are scared of cats'` would both produce the exact same vector, since they contain the same words.  

In the cell below, create a function that takes in a tokenized, cleaned song and returns a count vectorized representation of it as a Python dictionary.

**_Hint:_**  Consider using a `set()` since you'll need each unique word in the tokenized song! 

In [26]:
def count_vectorize(tokenized_song):
   unique_words = set(tokenized_song)
   word_counts = {word: 0 for word in unique_words}

   for word in tokenized_song:
        word_counts[word] += 1

   return word_counts

    

test_vectorized = count_vectorize(tokenized_test_song)
print(test_vectorized)

{'my': 16, 'on': 13, 'everybody': 4, 'oh': 17, 'kikiritikiki': 8, 'word': 8, 'fire': 6, 'baby': 24, 'yeah': 12, 'night': 24, 'two': 1, 'na': 18, 'see': 6, 'take': 1, 'bodys': 6, 'wrongs': 1, 'ungbambe': 4, 'party': 1, 'tell': 1, 'make': 1, 'ice': 6, 'go': 13, 'your': 12, 'unggudluke': 4, 'to': 1, 'us': 1, 'want': 6, 'ungdedele': 4, 'our': 1, 'ungbhasobhe': 4, 'of': 1, 'out': 12, 'drinks': 6, 'up': 16, 'dont': 1, 'wan': 18, 'we': 1, 'high': 16, 'me': 1, 'i': 30, 'somethings': 1, 'lives': 1, 'wrong': 1, 'gosh': 8, 'need': 6, 'and': 6, 'say': 4, 'you': 19, 'all': 25, 'right': 1, 'away': 2}


Great! You've just successfully vectorized your first text document! Now, let's look at a more advanced type of vectorization, TF-IDF!

### TF-IDF Vectorization

TF-IDF stands for **_Term Frequency, Inverse Document Frequency_**.  This is a more advanced form of vectorization that weighs each term in a document by how unique it is to the given document it is contained in, which allows us to summarize the contents of a document using a few key words.  If the word is used often in many other documents, it is not unique, and therefore probably not too useful if we wanted to figure out how this document is unique in relation to other documents. Conversely, if a word is used many times in a document, but rarely in all the other documents we are considering, then it is likely a good indicator for telling us that this word is important to the document in question.  

The formula TF-IDF uses to determine the weights of each term in a document is **_Term Frequency_** multiplied by **_Inverse Document Frequency_**. We just calculated our Term Frequency above with Count Vectorization!

Now that we have this, we can easily calculate _Inverse Document Frequency_. Note that this will need ALL of our documents (aka our songs), not just an individual document - so we'll put off testing this function for now.

In the cell below, complete a function that takes in a list of tokenized songs, with each item in the list being a clean, tokenized version of the song. The function should return a dictionary containing the inverse document frequency values for each word.  

The formula for Inverse Document Frequency is:  
<br>  
<br>
$$\large \text{IDF}(t) =  log_e(\frac{\text{Total Number of Documents}}{\text{Number of Documents with } t \text{ in it}})$$

In [27]:
import math

In [29]:
def inverse_document_frequency(list_of_token_songs):
    N = len(list_of_token_songs)

    frequency = {}
    for song_tokens in list_of_token_songs:
    
        unique_words_in_song = set(song_tokens)
        for word in unique_words_in_song:
            frequency[word] = frequency.get(word, 0) + 1

    idf_values = {}
    for word, df_count in frequency.items():
        # IDF formula: log_e(N / DF(t))
        idf_values[word] = math.log(N / df_count)

    return idf_values

inverse_document_frequency(test_vectorized)

{'y': 1.62924053973028,
 'm': 2.322387720290225,
 'o': 0.9873866535578852,
 'n': 1.1592369104845446,
 'b': 2.322387720290225,
 'v': 3.2386784521643803,
 'd': 1.7346010553881064,
 'r': 1.5339303599259553,
 'e': 1.0986122886681098,
 'h': 1.8523840910444898,
 'i': 1.62924053973028,
 't': 1.4469189829363254,
 'k': 2.322387720290225,
 'w': 1.8523840910444898,
 'f': 3.2386784521643803,
 'a': 1.292768303109067,
 'g': 1.4469189829363254,
 's': 1.62924053973028,
 'u': 1.62924053973028,
 'p': 3.2386784521643803,
 'l': 2.322387720290225,
 'c': 3.9318256327243257}

### Computing TF-IDF

Now that we can compute both Term Frequency and Inverse Document Frequency, computing an overall TF-IDF value is simple! All we need to do is multiply the two values.  

In the cell below, complete the `tf_idf()` function.  This function should take in a list of tokenized songs, just as the `inverse_document_frequency()` function did.  This function returns a new list of dictionaries, with each dictionary containing the tf-idf vectorized representation of a corresponding song document. You'll need to calculate the term frequency for each song using the `count_vectorize()` function we defined above.

**_NOTE:_** Each document should contain the full vocabulary of the entire combined corpus! So, even if a song doesn't have the word "kikiritikiki" (a vocalization in our test song), it should have a dictionary entry with that word as the key and `0` as the value.

In [32]:
def tf_idf(list_of_token_songs):
    idf = inverse_document_frequency(list_of_token_songs)

    tf_idf_vectors = []
    for tokenized_song in list_of_token_songs:
        total_words_in_song = len(tokenized_song)
        if total_words_in_song == 0:
            tf_idf_vectors.append({}) 
            continue
        raw_counts = count_vectorize(tokenized_song)

        current_song_tf_vector = {
            word: count / total_words_in_song
            for word, count in raw_counts.items()
        }

        current_song_tfidf_vector = {}
        for word, tf_score in current_song_tf_vector.items():
            idf_score = idf.get(word, 0.0)
            
            # TF-IDF is TF multiplied by IDF
            tfidf_score = tf_score * idf_score
            current_song_tfidf_vector[word] = tfidf_score

        tf_idf_vectors.append(current_song_tfidf_vector)

    return tf_idf_vectors

tf_idf(test_vectorized)

[{'y': 0.81462026986514, 'm': 1.1611938601451126},
 {'o': 0.4936933267789426, 'n': 0.5796184552422723},
 {'b': 0.25804308003224724,
  'v': 0.3598531613515978,
  'o': 0.10970962817309836,
  'd': 0.19273345059867847,
  'y': 0.36205345327339555,
  'r': 0.17043670665843946,
  'e': 0.24413606414846883},
 {'o': 0.4936933267789426, 'h': 0.9261920455222449},
 {'i': 0.81462026986514,
  't': 0.12057658191136045,
  'k': 0.7741292400967417,
  'r': 0.1278275299938296},
 {'w': 0.46309602276112244,
  'o': 0.2468466633894713,
  'd': 0.4336502638470266,
  'r': 0.3834825899814888},
 {'e': 0.27465307216702745,
  'r': 0.3834825899814888,
  'i': 0.40731013493257,
  'f': 0.8096696130410951},
 {'b': 1.1611938601451126, 'y': 0.40731013493257, 'a': 0.3231920757772668},
 {'a': 0.3231920757772668,
  'e': 0.27465307216702745,
  'y': 0.40731013493257,
  'h': 0.46309602276112244},
 {'i': 0.325848107946056,
  'g': 0.2893837965872651,
  't': 0.2893837965872651,
  'h': 0.370476818208898,
  'n': 0.23184738209690892},
 

### Vectorizing All Documents

Now that we've created all the necessary helper functions, we can load in all of our documents and run each through the vectorization pipeline we've just created.

In the cell below, complete the `main()` function.  This function should take in a list of file names (provided for you in the `filenames` list we created at the start), and then:

- Read in each document into a list of raw songs (where each song is a list of strings)
- Tokenize each document into a list of cleaned and tokenized songs
- Return a list of dictionaries vectorized using `tf-idf`, where each dictionary is a vectorized representation of each song 

In [38]:
def main(filenames):
    raw_songs = []
    cleaned_tokenized_songs = []

    for filename in filenames:
        full_path = os.path.join('data', filename)

        with open(full_path, 'r', encoding='utf-8') as f:
            song_lines = f.readlines()
            raw_songs.append(song_lines)

    for i, raw_song_lines in enumerate(raw_songs):
        cleaned_string = clean_song(raw_song_lines)
        tokenized_song = word_tokenize(cleaned_string)
        cleaned_tokenized_songs.append(tokenized_song)

    tfidf_vectors = tf_idf(cleaned_tokenized_songs)

    return tfidf_vectors

tf_idf_all_docs = main(filenames)
tf_idf_all_docs

[{'everyone': 0.021060229509091883,
  'trinity': 0.027399990306896257,
  'my': 0.0052534501962494436,
  'spent': 0.009133330102298753,
  'pristine': 0.009133330102298753,
  'rolled': 0.007020076503030628,
  'walked': 0.009133330102298753,
  'japan': 0.009133330102298753,
  'oh': 0.004226507198536252,
  'ventura': 0.009133330102298753,
  'queens': 0.009133330102298753,
  'sea': 0.027399990306896257,
  'herb': 0.009133330102298753,
  'passage': 0.027399990306896257,
  'it': 0.006803157052262493,
  'letters': 0.009133330102298753,
  'runnin': 0.012679521595608754,
  'forever': 0.014040153006061255,
  'time': 0.0036453475655830515,
  'every': 0.008453014397072503,
  'big': 0.004226507198536252,
  'an': 0.004906822903762501,
  'teach': 0.027399990306896257,
  'ear': 0.009133330102298753,
  'leaf': 0.009133330102298753,
  'that': 0.002972907246910517,
  'cross': 0.009133330102298753,
  'beneath': 0.007020076503030628,
  'hands': 0.0057839023929447606,
  'valentines': 0.009133330102298753,
  

### Level Up: Visualizing our Vectorizations (Optional)

Now that we have a tf-idf representation of each document, we can move on to the fun part -- visualizing everything!

In the cell below, examine our dictionaries to figure out how many dimensions our dataset has. 

**_HINT_**: Remember that every word is its own dimension!

In [ ]:
num_dims = None
print(f"Number of Dimensions: {num_dims}")

There are too many dimensions for us to visualize! In order to make it understandable to human eyes, we'll need to reduce it to 2 or 3 dimensions.  

To do this, we'll use a technique called **_t-SNE_** (short for _t-Stochastic Neighbors Embedding_).  This is too complex for us to code ourselves, so we'll make use of scikit-learn's implementation of it.  

First, we need to pull the words out of the dictionaries stored in `tf_idf_all_docs` so that only the values remain, and store them in lists instead of dictionaries.  This is because the t-SNE only works with array-like objects, not dictionaries.  

In the cell below, create a list of lists that contains a list representation of the values of each of the dictionaries stored in `tf_idf_all_docs`.  The same structure should remain -- e.g. the first list should contain only the values that were in the first dictionary in `tf_idf_all_docs`, and so on. 

In [ ]:
tf_idf_vals_list = None

tf_idf_vals_list[0][:10]

Now that we have only the values, we can use the `TSNE()` class from `sklearn` to transform our data appropriately. In the cell below, instantiate `TSNE()` with the following arguments:
- `n_components=3` (so we can compare 2 vs 3 components when graphing)
- `perplexity=19` (the highest number of neighbors explored given the size of our dataset)
- `learning_rate=200` (a higher learning rate than using 'auto', to avoid getting stuck in a local minimum)
- `init='random'` (so SKLearn will randomize the initialization)
- `random_state=13` (so that random initialization won't be TOO random)

Then, use the created object's `.fit_transform()` method to transform the data stored in `tf_idf_vals_list` into 3-dimensional data.  Then, inspect the newly transformed data to confirm that it has the correct dimensionality. 

In [ ]:
t_sne_object_3d = None

transformed_data_3d = t_sne_object_3d.fit_transform(np.array(tf_idf_vals_list))
transformed_data_3d

We'll also want to check out how the visualization looks in 2d.  Repeat the process above, but this time, instantiate `TSNE()` with 2 components instead of 3.  Again, use `.fit_transform()` to transform the data and store it in the variable below, and then inspect it to confirm the transformed data has only 2 dimensions. 

In [ ]:
t_sne_object_2d = None

transformed_data_2d = t_sne_object_2d.fit_transform(np.array(tf_idf_vals_list))
transformed_data_2d

Now, let's visualize everything!  Run the cell below to view both 3D and 2D visualizations of the songs.

In [ ]:
kendrick_3d = transformed_data_3d[:10]
k3_x = [i[0] for i in kendrick_3d]
k3_y = [i[1] for i in kendrick_3d]
k3_z = [i[2] for i in kendrick_3d]

garth_3d = transformed_data_3d[10:]
g3_x = [i[0] for i in garth_3d]
g3_y = [i[1] for i in garth_3d]
g3_z = [i[2] for i in garth_3d]

fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(k3_x, k3_y, k3_z, c='b', s=60, label='Kendrick')
ax.scatter(g3_x, g3_y, g3_z, c='red', s=60, label='Garth')
ax.view_init(40,10)
ax.legend()
plt.show()

kendrick_2d = transformed_data_2d[:10]
k2_x = [i[0] for i in kendrick_2d]
k2_y = [i[1] for i in kendrick_2d]

garth_2d = transformed_data_2d[10:]
g2_x = [i[0] for i in garth_2d]
g2_y = [i[1] for i in garth_2d]

fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(222)
ax.scatter(k2_x, k2_y, c='b', label='Kendrick')
ax.scatter(g2_x, g2_y, c='red', label='Garth')
ax.legend()
plt.show()

Interesting! Take a crack at interpreting these graphs by answering the following questions below:

What does each graph mean? Do you find one graph more informative than the other? Do you think that this method shows us discernable differences between Kendrick Lamar songs and Garth Brooks songs?  Use the graphs and your understanding of TF-IDF to support your answer.  

In [ ]:
'''
WRITE YOUR ANSWER HERE
'''

## Summary

In this lab, you learned how to: 
* Tokenize a corpus of words and identify the different choices to be made while parsing them 
* Use a count vectorization strategy to create a bag of words
* Use TF-IDF vectorization with multiple documents to identify words that are important/unique to certain documents
* Visualize and compare vectorized text documents